In [1]:
# -*- coding: utf-8 -*-
import os as os
import pandas as pd
import numpy as np
import pickle
import pyodbc
import glob
import csv
import random
# import preprocessing
import time
from sklearn.ensemble import RandomForestClassifier
# from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier #整合(可分配占比)
# from sklearn import cross_validation #交叉驗證資料準確度(這個以廢棄)
# from sklearn.cross_validation import cross_val_score #交叉驗證資料準確度(這個以廢棄)
from sklearn.model_selection import cross_val_score #交叉驗證資料準確度
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

print("【訊息】目前的工作目錄:",os.getcwd())

【訊息】目前的工作目錄: D:\Mypython


In [2]:
traindata = []
trainlabel = []
datafile=pd.read_csv(r'D:\Mypython\train.csv', sep=',', index_col=0, header=0)
print(list(datafile.columns))    #觀看輸入資料的欄位
datafile['target'] = datafile['target'].str[-1].astype(int)  #將 class1 , class2...class9 轉換成 1 , 2...9
for i in range(1,len(datafile)):   #6萬筆資料的輸入
    l = []
    l = datafile.loc[i,datafile.columns[0:93]] 
    traindata.append(l)     #每筆放入 feat_1 , feat_2 ..... feat_93
    trainlabel.append(datafile.loc[i,datafile.columns[-1]])  #把每筆的 target 放入
traindata = np.array(traindata,dtype="float") #轉化成陣列  
trainlabel = np.array(trainlabel,dtype="int") #轉化成陣列
# trainlabel = np.array(trainlabel,dtype="str") #轉化成陣列，但轉字串
mean = traindata.mean(axis=0)   #平均
# axis = 0：壓縮行，對各列求均值，返回 1 * n 矩陣
# axis = 1 ：壓縮列，對各行求均值，返回 m * 1 矩陣
std = traindata.std(axis=0)     #標準差
traindata = (traindata - mean)/std #正規化


['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48', 'feat_49', 'feat_50', 'feat_51', 'feat_52', 'feat_53', 'feat_54', 'feat_55', 'feat_56', 'feat_57', 'feat_58', 'feat_59', 'feat_60', 'feat_61', 'feat_62', 'feat_63', 'feat_64', 'feat_65', 'feat_66', 'feat_67', 'feat_68', 'feat_69', 'feat_70', 'feat_71', 'feat_72', 'feat_73', 'feat_74', 'feat_75', 'feat_76', 'feat_77', 'feat_78', 'feat_79', 'feat_80', 'feat_81', 'feat_82', 'feat_83', 'feat_84', 'feat_85', 'feat_86', 'feat_87', 'feat_88', 'feat_89', 'feat_90', 'feat_91', 'feat_9

In [3]:
randomIndex = [i for i in range(len(trainlabel))]  # 多少筆資料
random.shuffle(randomIndex)   #亂數洗牌 ， 以免每筆資料有連貫
traindata = traindata[randomIndex]
trainlabel = trainlabel[randomIndex]

In [20]:
testdata = []
datafile2=pd.read_csv(r'D:\Mypython\test.csv', sep=',', index_col=0, header=0)
for i in range(1,len(datafile2)+1):   #6萬筆資料的輸入
    l = []
    l = datafile2.loc[i,datafile2.columns[0:93]] 
    testdata.append(l)
testdata = np.array(testdata,dtype="float")
mean = testdata.mean(axis=0) 
std = testdata.std(axis=0)
testdata = (testdata - mean)/std 

In [21]:
print(len(testdata))

144368


In [22]:
rfClf = RandomForestClassifier(criterion='gini', n_jobs=-1, n_estimators=500, max_features=18, min_samples_split=5, max_depth=46, class_weight=None)
RF=rfClf.fit(traindata,trainlabel)  #餵進資料進行訓練，並模型建置
test_pred_label = rfClf.predict_proba(testdata)  # 返回预测属于某標籤的概率 
print(rfClf.predict(testdata)) # 返回预测標籤
print(list(test_pred_label[1]))

scores = cross_val_score(rfClf,traindata,trainlabel,cv=5,scoring='accuracy') #交叉驗證分5組
print("每組評分:",np.floor(scores*100))
print("模型評分:",int(scores.mean()*100),"% 的準確率")


[4 6 6 ... 2 2 2]
[0.019942857142857144, 0.04380952380952381, 0.040328571428571436, 0.030452380952380953, 0.0066, 0.47808730158730167, 0.021857142857142856, 0.34785555555555553, 0.011066666666666667]
每組評分: [81. 80. 81. 81. 81.]
模型評分: 81 % 的準確率


In [8]:
knnClf = KNeighborsClassifier(n_neighbors=128, n_jobs=-1)
KN=knnClf.fit(traindata,trainlabel)  #餵進資料進行訓練，並模型建置
test_pred_label = knnClf.predict_proba(testdata)  # 返回预测属于某標籤的概率 
print(knnClf.predict(testdata)) # 返回预测標籤
print(list(test_pred_label[1]))

scores = cross_val_score(knnClf,traindata,trainlabel,cv=5,scoring='accuracy') #交叉驗證分5組
print("每組評分:",np.floor(scores*100))
print("模型評分:",int(scores.mean()*100),"% 的準確率")


[2 6 6 ... 3 3 2]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.984375, 0.0, 0.015625, 0.0]
每組評分: [73. 73. 73. 73. 73.]
模型評分: 73 % 的準確率


In [23]:
LRClf =  LogisticRegression(penalty='l2', solver='lbfgs', n_jobs=-1, C=0.5, class_weight=None, multi_class='multinomial')
LR=LRClf.fit(traindata,trainlabel)  #餵進資料進行訓練，並模型建置
test_pred_label = LRClf.predict_proba(testdata)  # 返回预测属于某標籤的概率 
print(LRClf.predict(testdata)) # 返回预测標籤
print(list(test_pred_label[1]))

[4 6 6 ... 3 2 2]
[0.00030319026173244713, 7.20654531739938e-05, 5.224655157781561e-07, 5.619158647183859e-08, 1.1892818690111713e-05, 0.699755188893166, 0.00041800279395069046, 0.29831063112136963, 0.0011284500008147873]


In [28]:
XGClf =XGBClassifier(base_score=0.5, colsample_bylevel=1,
                             colsample_bytree=0.9, gamma=1,
                             learning_rate=1, max_delta_step=0, max_depth=30,
                             min_child_weight=7, n_estimators=6, nthread=-1,
                             objective='multi:softprob', reg_alpha=0, reg_lambda=1,
                             scale_pos_weight=1, subsample=1)
XG = XGClf.fit(traindata,trainlabel)  #餵進資料進行訓練，並模型建置
test_pred_label = XGClf.predict_proba(testdata)  # 返回预测属于某標籤的概率 
print(XGClf.predict(testdata)) # 返回预测標籤
print(list(test_pred_label[1]))

[4 6 6 ... 2 4 3]
[0.07055033, 0.0031603281, 0.0039660516, 0.012828649, 0.0020049645, 0.5785077, 0.009150462, 0.31168106, 0.008150504]


In [29]:
np.savetxt(r"D:\MyPython\Kaggle_XGBClassifier.csv", test_pred_label, delimiter=",")

In [25]:
# model ensemble (模型整合)
eclf = VotingClassifier(estimators=[('rf', RF), ('lr', LR),('xgb',XG)],voting='soft', weights=[3 , 1 , 2])
eclf.fit(traindata,trainlabel)
test_pred_label = eclf.predict_proba(testdata)
print(eclf.predict(testdata)) # 返回预测標籤
print(list(test_pred_label[1]))


[4 6 6 ... 2 4 3]
[0.0385613559904145, 0.02316505679633873, 0.022901469363451003, 0.015529003483603898, 0.004632208401679411, 0.5347443116394349, 0.013781725969256387, 0.33635379251459785, 0.010331100133220483]


In [26]:
print(len(test_pred_label))

144368


In [27]:
np.savetxt(r"D:\MyPython\Kaggle_VotingClassifier.csv", test_pred_label, delimiter=",")